# Experiment 7 - Use a Classifier and Create an API to it

## Import Libraries

In [3]:
from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
import pickle

import numpy as np
import requests
import json

from flask import Flask, jsonify
from flask_restful import Api, Resource, reqparse

## Train the Model

In [4]:
# Load Iris data
iris = load_iris()

# Split into train and test sets
X_train, X_test, y_train, y_test = train_test_split(iris['data'], iris['target'], random_state=12)

# Train the model
clf = RandomForestClassifier(random_state=12)
clf.fit(X_train, y_train)

# Make prediction on the test set
y_predict = clf.predict(X_test)
print(y_predict)

# Save model
with open('model.pickle', 'wb') as f:
    pickle.dump(clf, f)

[0 2 0 1 2 2 2 0 2 0 1 0 0 0 1 2 2 1 0 1 0 1 2 1 0 2 2 1 0 0 0 1 2 0 2 0 1
 1]


## Test the Model

In [ ]:
# Load data
iris = load_iris()

# Split into train and test sets using the same random state
X_train, X_test, y_train, y_test = train_test_split(iris['data'], iris['target'], random_state=12)

# Serialize the data into json and send the request to the model
payload = {'data': json.dumps(X_test.tolist())}
y_predict = requests.post('http://127.0.0.1:5000/iris', data=payload).json()

# Make array from the list
y_predict = np.array(y_predict)
print(y_predict)

## Deploy the Model to a RESTfulAPI

In [6]:
app = Flask(__name__)
api = Api(app)

# Create parser for the payload data
parser = reqparse.RequestParser()
parser.add_argument('data')

# Define how the api will respond to the post requests
class IrisClassifier(Resource):
    def post(self):
        args = parser.parse_args()
        X = np.array(json.loads(args['data']))
        prediction = model.predict(X)
        return jsonify(prediction.tolist())

api.add_resource(IrisClassifier, '/iris')

if __name__ == '__main__':
    # Load model
    with open('model.pickle', 'rb') as f:
        model = pickle.load(f)

    app.run(debug=True)

 * Serving Flask app '__main__'
 * Debug mode: on


 * Running on http://127.0.0.1:5000
Press CTRL+C to quit
 * Restarting with watchdog (inotify)
/home/volt/.local/lib/python3.10/site-packages/traitlets/traitlets.py:2547: FutureWarning: Supporting extra quotes around strings is deprecated in traitlets 5.0. You can use 'hmac-sha256' instead of '"hmac-sha256"' if you require traitlets >=5.
  warn(
/home/volt/.local/lib/python3.10/site-packages/traitlets/traitlets.py:2498: FutureWarning: Supporting extra quotes around Bytes is deprecated in traitlets 5.0. Use 'ec5dfbcb-f82d-4d70-b78d-fa520930b867' instead of 'b"ec5dfbcb-f82d-4d70-b78d-fa520930b867"'.
  warn(
Traceback (most recent call last):
  File "/home/volt/.local/lib/python3.10/site-packages/ipykernel_launcher.py", line 17, in <module>
    app.launch_new_instance()
  File "/home/volt/.local/lib/python3.10/site-packages/traitlets/config/application.py", line 1040, in launch_instance
    app.initialize(argv)
  File "/home/volt/.local/lib/python3.10/site-packages/traitlets/config/applic

SystemExit: 1

/home/volt/.local/lib/python3.10/site-packages/IPython/core/interactiveshell.py:3441: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)
